### [Retrieving a Context Window Around a Sentence](https://haystack.deepset.ai/tutorials/42_sentence_window_retriever)

In [1]:
#! Solo es para haystack sepa que tutorial se esta ejecutando
from haystack.telemetry import tutorial_running
tutorial_running(42)

### 1. Getting started with Sentence-Window Retrieval

In [3]:
from haystack import Document
from haystack.components.preprocessors import DocumentSplitter
splitter = DocumentSplitter(split_length=1, split_overlap=0, split_by="sentence")

text = ("Paul fell asleep to dream of an Arrakeen cavern, silent people all around  him moving in the dim light "
        "of glowglobes. It was solemn there and like a cathedral as he listened to a faint sound—the "
        "drip-drip-drip of water. Even while he remained in the dream, Paul knew he would remember it upon "
        "awakening. He always remembered the dreams that were predictions. The dream faded. Paul awoke to feel "
        "himself in the warmth of his bed—thinking thinking. This world of Castle Caladan, without play or "
        "companions his own age,  perhaps did not deserve sadness in farewell. Dr Yueh, his teacher, had "
        "hinted  that the faufreluches class system was not rigidly guarded on Arrakis. The planet sheltered "
        "people who lived at the desert edge without caid or bashar to command them: will-o’-the-sand people "
        "called Fremen, marked down on no  census of the Imperial Regate.")

doc = Document(content=text)
docs = splitter.run([doc])
docs

{'documents': [Document(id=8db98f03cd338b83a8b28172ebf03c77fb481c17053e792193033003daeffc8a, content: 'Paul fell asleep to dream of an Arrakeen cavern, silent people all around  him moving in the dim lig...', meta: {'source_id': 'b56504f244b7b650096b14d678bc82f3d7fe240bb135361c6a23a14c4b809596', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0}),
  Document(id=db478194d25cedfb818f04c4bd22a5d4c058f6cddcced2461445d82f7feaf932, content: ' It was solemn there and like a cathedral as he listened to a faint sound—the drip-drip-drip of wate...', meta: {'source_id': 'b56504f244b7b650096b14d678bc82f3d7fe240bb135361c6a23a14c4b809596', 'page_number': 1, 'split_id': 1, 'split_idx_start': 117}),
  Document(id=5d093b6ec1a4bdc7e75f033ae0b570e237053213a09b42a56ad815b4d118943d, content: ' Even while he remained in the dream, Paul knew he would remember it upon awakening.', meta: {'source_id': 'b56504f244b7b650096b14d678bc82f3d7fe240bb135361c6a23a14c4b809596', 'page_number': 1, 'split_id': 2, 'spli

In [5]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.document_stores.types import DuplicatePolicy

doc_store = InMemoryDocumentStore()
doc_store.write_documents(docs['documents'], policy=DuplicatePolicy.OVERWRITE)
doc_store.filter_documents()

[Document(id=8db98f03cd338b83a8b28172ebf03c77fb481c17053e792193033003daeffc8a, content: 'Paul fell asleep to dream of an Arrakeen cavern, silent people all around  him moving in the dim lig...', meta: {'source_id': 'b56504f244b7b650096b14d678bc82f3d7fe240bb135361c6a23a14c4b809596', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0}),
 Document(id=db478194d25cedfb818f04c4bd22a5d4c058f6cddcced2461445d82f7feaf932, content: ' It was solemn there and like a cathedral as he listened to a faint sound—the drip-drip-drip of wate...', meta: {'source_id': 'b56504f244b7b650096b14d678bc82f3d7fe240bb135361c6a23a14c4b809596', 'page_number': 1, 'split_id': 1, 'split_idx_start': 117}),
 Document(id=5d093b6ec1a4bdc7e75f033ae0b570e237053213a09b42a56ad815b4d118943d, content: ' Even while he remained in the dream, Paul knew he would remember it upon awakening.', meta: {'source_id': 'b56504f244b7b650096b14d678bc82f3d7fe240bb135361c6a23a14c4b809596', 'page_number': 1, 'split_id': 2, 'split_idx_start': 21

In [13]:
from haystack.components.retrievers import SentenceWindowRetriever

retriever = SentenceWindowRetriever(document_store=doc_store, window_size=2)
result = retriever.run(retrieved_documents=[docs['documents'][2]])
result

{'context_windows': ['Paul fell asleep to dream of an Arrakeen cavern, silent people all around  him moving in the dim light of glowglobes. It was solemn there and like a cathedral as he listened to a faint sound—the drip-drip-drip of water. Even while he remained in the dream, Paul knew he would remember it upon awakening. He always remembered the dreams that were predictions. The dream faded.'],
 'context_documents': [[Document(id=8db98f03cd338b83a8b28172ebf03c77fb481c17053e792193033003daeffc8a, content: 'Paul fell asleep to dream of an Arrakeen cavern, silent people all around  him moving in the dim lig...', meta: {'source_id': 'b56504f244b7b650096b14d678bc82f3d7fe240bb135361c6a23a14c4b809596', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0}),
   Document(id=db478194d25cedfb818f04c4bd22a5d4c058f6cddcced2461445d82f7feaf932, content: ' It was solemn there and like a cathedral as he listened to a faint sound—the drip-drip-drip of wate...', meta: {'source_id': 'b56504f244b7b65009

In [14]:
#context_windows:una lista de cadenas que contienen las ventanas de contexto alrededor de la oración coincidente.
result['context_windows']

['Paul fell asleep to dream of an Arrakeen cavern, silent people all around  him moving in the dim light of glowglobes. It was solemn there and like a cathedral as he listened to a faint sound—the drip-drip-drip of water. Even while he remained in the dream, Paul knew he would remember it upon awakening. He always remembered the dreams that were predictions. The dream faded.']

In [15]:
#context_documents:una lista de listas de Documentobjetos que contienen las ventanas de contexto alrededor de la oración coincidente.
result['context_documents']

[[Document(id=8db98f03cd338b83a8b28172ebf03c77fb481c17053e792193033003daeffc8a, content: 'Paul fell asleep to dream of an Arrakeen cavern, silent people all around  him moving in the dim lig...', meta: {'source_id': 'b56504f244b7b650096b14d678bc82f3d7fe240bb135361c6a23a14c4b809596', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0}),
  Document(id=db478194d25cedfb818f04c4bd22a5d4c058f6cddcced2461445d82f7feaf932, content: ' It was solemn there and like a cathedral as he listened to a faint sound—the drip-drip-drip of wate...', meta: {'source_id': 'b56504f244b7b650096b14d678bc82f3d7fe240bb135361c6a23a14c4b809596', 'page_number': 1, 'split_id': 1, 'split_idx_start': 117}),
  Document(id=5d093b6ec1a4bdc7e75f033ae0b570e237053213a09b42a56ad815b4d118943d, content: ' Even while he remained in the dream, Paul knew he would remember it upon awakening.', meta: {'source_id': 'b56504f244b7b650096b14d678bc82f3d7fe240bb135361c6a23a14c4b809596', 'page_number': 1, 'split_id': 2, 'split_idx_start':

### 2. Create a Keyword Retrieval Pipeline with Sentence-Window Retrieval

In [18]:
from typing import List
import csv
from haystack import Document

def read_documents(file: str) -> List[Document]:
    with open(file, "r") as file:
        reader = csv.reader(file, delimiter="\t")
        next(reader, None)  # skip the headers
        documents = []
        for row in reader:
            category = row[0].strip()
            title = row[2].strip()
            text = row[3].strip()
            documents.append(Document(content=text, meta={"category": category, "title": title}))

    return documents


In [21]:
from pathlib import Path
import requests

doc = requests.get('https://raw.githubusercontent.com/amankharwal/Website-data/master/bbc-news-data.csv')

datafolder = Path('data')
datafolder.mkdir(exist_ok=True)
with open(datafolder/'bbc-news-data.csv', 'wb') as f:
    for chunk in doc.iter_content(512):
        f.write(chunk)

docs = read_documents("data/bbc-news-data.csv")
len(docs)

2225

In [26]:
## Indexar documentos
from haystack import Document, Pipeline
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.writers import DocumentWriter
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.document_stores.types import DuplicatePolicy

doc_store = InMemoryDocumentStore()

indexing_pipeline = Pipeline()
indexing_pipeline.add_component("splitter", DocumentSplitter(split_length=1, split_overlap=0, split_by="sentence"))
indexing_pipeline.add_component("writer", DocumentWriter(document_store=doc_store, policy=DuplicatePolicy.OVERWRITE))

indexing_pipeline.connect("splitter", "writer")
indexing_pipeline.run({"documents":docs})

{'writer': {'documents_written': 44186}}

In [23]:
## Construir una secuencia de recuperación de oraciones por ventana
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.retrievers import SentenceWindowRetriever

sentence_window_pipeline = Pipeline()

sentence_window_pipeline.add_component("bm25_retriever", InMemoryBM25Retriever(document_store=doc_store))
sentence_window_pipeline.add_component("sentence_window__retriever", SentenceWindowRetriever(doc_store, window_size=2))

sentence_window_pipeline.connect("bm25_retriever.documents", "sentence_window__retriever.retrieved_documents")

🚅 Components
  - bm25_retriever: InMemoryBM25Retriever
  - sentence_window__retriever: SentenceWindowRetriever
🛤️ Connections
  - bm25_retriever.documents -> sentence_window__retriever.retrieved_documents (List[Document])

In [ ]:
## Poniendo todo junto
result = sentence_window_pipeline.run(data={'bm25_retriever': {'query': "phishing attacks", "top_k": 1}}, include_outputs_from={'bm25_retriever'})

In [28]:
result['bm25_retriever']['documents']

[Document(id=57766497f35c7ebef5c49e754b8df41a8df3d5df3e46bc595807d7420d7cef8e, content: ' The Anti-Phishing Working group reported that the number of phishing attacks against new targets wa...', meta: {'category': 'tech', 'title': 'Cyber crime booms in 2004', 'source_id': '5c81f8cbd6c9c07819bf60e484489fe0af9e6626591ec77066701cb856fb3b33', 'page_number': 1, 'split_id': 12, 'split_idx_start': 1520}, score: 17.74585935028894)]

In [29]:
result['sentence_window__retriever']['context_windows']

['"  In particular, phishing attacks, which typically use fake versions of bank websites to grab login details of customers, boomed during 2004. Web portal Lycos Europe reported a 500% increase in the number of phishing e-mail messages it was catching. The Anti-Phishing Working group reported that the number of phishing attacks against new targets was growing at a rate of 30% or more per month. Those who fall victim to these attacks can find that their bank account has been cleaned out or that their good name has been ruined by someone stealing their identity. This change in the ranks of virus writers could mean the end of the mass-mailing virus which attempts to spread by tricking people into opening infected attachments on e-mail messages.']

In [30]:
result['sentence_window__retriever']['context_documents']

[[Document(id=905582f4a147cae72b90223e433db5986c4ff46d8c8a325fe56ea3cfbecff742, content: '"  In particular, phishing attacks, which typically use fake versions of bank websites to grab login...', meta: {'category': 'tech', 'title': 'Cyber crime booms in 2004', 'source_id': '5c81f8cbd6c9c07819bf60e484489fe0af9e6626591ec77066701cb856fb3b33', 'page_number': 1, 'split_id': 10, 'split_idx_start': 1270}),
  Document(id=91f6969683e714cddf3ef4816616176d7e467bb7756eb4051f0aa5f15e7bcabd, content: ' Web portal Lycos Europe reported a 500% increase in the number of phishing e-mail messages it was c...', meta: {'category': 'tech', 'title': 'Cyber crime booms in 2004', 'source_id': '5c81f8cbd6c9c07819bf60e484489fe0af9e6626591ec77066701cb856fb3b33', 'page_number': 1, 'split_id': 11, 'split_idx_start': 1412}),
  Document(id=57766497f35c7ebef5c49e754b8df41a8df3d5df3e46bc595807d7420d7cef8e, content: ' The Anti-Phishing Working group reported that the number of phishing attacks against new targets wa...'